In [1]:
import pandas as pd
import csv
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

df_2012_train = pd.read_csv('df_2012_train.csv') 
df_2012_train = df_2012_train.set_index('fips')
df_2012_test = pd.read_csv('df_2012_test.csv') 
df_2012_test = df_2012_test.set_index('fips')
df_2016_train = pd.read_csv('df_2016_train.csv') 
df_2016_train = df_2016_train.set_index('county_name')
df_2016_test = pd.read_csv('df_2016_test.csv') 
df_2016_test = df_2016_test.set_index('county_name')

In [2]:
df_2012_train.head(2)

,state_abbreviation,county_name,diff_2012,land_area,population,perc_female,perc_u5,perc_o18,perc_o65,perc_2races,...,hc_value_1k,food_value_1k,median_income,per_cap_income,perc_pov,num_establishments,annual_payroll,vet_num_firms,fem_num_firms,min_num_firms
fips,,,,,,,,,,,,,,,,,,,,,
50011,VT,Franklin County,0.235485,0.607936,0.554590,0.852168,0.752126,0.454698,0.679059,0.334973,...,0.618064,0.598671,0.573469,0.538418,0.619473,0.426309,0.575188,0.366486,0.414746,0.166732
46093,SD,Meade County,-0.428453,0.800869,0.501383,0.814002,0.806294,0.452165,0.683144,0.451818,...,0.146813,0.044439,0.491513,0.502540,0.659653,0.397049,0.523883,0.330485,0.352493,0.177449


In [3]:
df_2012_train_x = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y = df_2012_train[['diff_2012']]

df_2016_train_x = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y = df_2016_train[['diff_2016']]

df_2012_test_x = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y = df_2012_test[['diff_2012']]

df_2016_test_x = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y = df_2016_test[['diff_2016']]

full_train_x = [df_2012_train_x, df_2016_train_x]
full_train_x = pd.concat(full_train_x)
df_2012_train_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y = [df_2012_train_y, df_2016_train_y]
full_train_y = pd.concat(full_train_y)

full_test_x = [df_2012_test_x, df_2016_test_x]
full_test_x = pd.concat(full_test_x)
df_2012_test_y.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y = [df_2012_test_y, df_2016_test_y]
full_test_y = pd.concat(full_test_y)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [4]:
#2012 FEATURE SELECTION

from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel

rgr = RandomForestRegressor(n_estimators=100, max_features=.33, max_depth=None, min_samples_split=3, criterion='mse', random_state = 31)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2012_train_x, df_2012_train_y)

selectedFeatures = featureSelection.transform(df_2012_train_x)
#selectedFeatures

fs = df_2012_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2012_train_x = df_2012_train_x[keep]
df_2012_test_x = df_2012_test_x[keep]




C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']


In [5]:
#2016 feature selection

rgr = RandomForestRegressor(n_estimators=100, max_features=.33, max_depth=None, min_samples_split=3, criterion='mse', random_state = 31)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(df_2016_train_x, df_2016_train_y)

selectedFeatures = featureSelection.transform(df_2016_train_x)
#selectedFeatures

fs = df_2016_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

df_2016_train_x = df_2016_train_x[keep]
df_2016_test_x = df_2016_test_x[keep]

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


['population', 'perc_white', 'perc_black', 'perc_asian', 'perc_white_not_hisp', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'annual_payroll', 'fem_num_firms', 'min_num_firms']


In [5]:
rgr = RandomForestRegressor(n_estimators=100, max_features=.33, max_depth=None, min_samples_split=3, criterion='mse', random_state = 31)
featureSelection = SelectFromModel(rgr, prefit=False, threshold='mean', max_features=None)
featureSelection.fit(full_train_x, full_train_y)

selectedFeatures = featureSelection.transform(full_train_x)
#selectedFeatures

fs = full_train_x.columns[featureSelection.get_support()]
#remaining features
keep = []
for i in range(len(fs)):
    keep.append(fs[i])
print(keep)

full_train_x = full_train_x[keep]
full_test_x = full_test_x[keep]

C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\feature_selection\from_model.py:197: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.estimator_.fit(X, y, **fit_params)


['population', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'annual_payroll', 'min_num_firms']


In [6]:
rf = RandomForestRegressor(random_state = 32)
pprint(rf.get_params())

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 32,
 'verbose': 0,
 'warm_start': False}


In [7]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10, 231, 452, 673, 894, 1115, 1336, 1557, 1778, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2, random_state=31, n_jobs = -1)
# Fit the random search model
rf_random.fit(df_2016_train_x, df_2016_train_y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("rmse:",rmse)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

print("BASE MODEL")
base_model = RandomForestRegressor(n_estimators = 10, random_state = 31)
base_model.fit(df_2016_train_x, df_2016_train_y)
base_accuracy = evaluate(base_model, df_2016_test_x, df_2016_test_y)

BASE MODEL


C:\Users\sajor\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


rmse: 0.15788562608273382
r2: 0.7035345036128222


In [15]:
rf_random.best_params_

{'n_estimators': 1115,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': False}

In [16]:
print('best random')
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, df_2012_test_x, df_2012_test_y)

best random
rmse: 0.14441238530421233
r2: 0.7519385036900215


In [11]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [False],
    'max_depth': [80, 90, 100],
    'max_features': ['sqrt'],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [3, 5, 7],
    'n_estimators': [900, 1100, 1300]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [12]:
# Fit the grid search to the data
grid_search.fit(df_2012_train_x, df_2012_train_y)
grid_search.best_params_



Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 26.9min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GRID Search


NameError: name 'evaluate' is not defined

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    print("mse:", mse)
    print("rmse:",rmse)
    mae = mean_absolute_error(test_labels, predictions)
    print("mae:", mae)
    r2 = explained_variance_score(test_labels, predictions)
    print("r2:",r2)

In [17]:
df_2012_train_x2 = df_2012_train.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_train_y2 = df_2012_train[['diff_2012']]

df_2016_train_x2 = df_2016_train.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_train_y2 = df_2016_train[['diff_2016']]

df_2012_test_x2 = df_2012_test.drop(columns = ['state_abbreviation', 'county_name', 'diff_2012'])
df_2012_test_y2 = df_2012_test[['diff_2012']]

df_2016_test_x2 = df_2016_test.drop(columns = ['state_abbreviation', 'fips', 'diff_2016'])
df_2016_test_y2 = df_2016_test[['diff_2016']]

full_train_x2 = [df_2012_train_x2, df_2016_train_x2]
full_train_x2 = pd.concat(full_train_x2)
df_2012_train_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_train_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_train_y2 = [df_2012_train_y2, df_2016_train_y2]
full_train_y2 = pd.concat(full_train_y2)

full_test_x2 = [df_2012_test_x2, df_2016_test_x2]
full_test_x2 = pd.concat(full_test_x2)
df_2012_test_y2.rename(columns={'diff_2012':'diff'}, inplace=True)
df_2016_test_y2.rename(columns={'diff_2016':'diff'}, inplace=True)

full_test_y2 = [df_2012_test_y2, df_2016_test_y2]
full_test_y2 = pd.concat(full_test_y2)

C:\Users\sajor\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [27]:
test_x = df_2016_test_x2[['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']]
full_x = full_test_x2[['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']]



print('2012 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, df_2012_test_x, df_2012_test_y)
print('2012 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test_x, df_2016_test_y2)
print('2012 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2012 GRID Search on 2012
mse: 0.02058647512441681
rmse: 0.14347987707137475
mae: 0.10995467835730538
r2: 0.7551305171702616
2012 GRID Search on 2016
mse: 0.030519936420614912
rmse: 0.1746995604476866
mae: 0.14006175938539775
r2: 0.6956666288329436
2012 GRID Search on Full
mse: 0.02555320577251586
rmse: 0.15985370115363567
mae: 0.12500821887135155
r2: 0.7258956489113116


In [30]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 1100}

In [47]:
fts = ['population', 'perc_o18', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'perc_oo', 'monthly_costs_m', 'monthly_costs_nm', 'num_nonemployers', 'man_value_1k', 'wholesale_value_1k', 'num_establishments', 'annual_payroll', 'min_num_firms']
list(zip(fts,list(grid_search.best_estimator_.feature_importances_)))

[('population', 0.061909213924437226),
 ('perc_o18', 0.05631558585998506),
 ('perc_white', 0.12160769408996623),
 ('perc_black', 0.07781969028592962),
 ('perc_white_not_hisp', 0.14967089259828278),
 ('perc_oo', 0.051270007476039235),
 ('monthly_costs_m', 0.05426902818147784),
 ('monthly_costs_nm', 0.11496942935065055),
 ('num_nonemployers', 0.051320087499403554),
 ('man_value_1k', 0.05421949050145596),
 ('wholesale_value_1k', 0.05485213509794516),
 ('num_establishments', 0.05233131831980832),
 ('annual_payroll', 0.05051161560396857),
 ('min_num_firms', 0.048933811210649596)]

In [48]:
# Fit the grid search to the data
grid_search.fit(df_2016_train_x, df_2016_train_y)
grid_search.best_params_


Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 26.2min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 1100}

In [49]:
lst = ['population', 'perc_white', 'perc_black', 'perc_asian', 'perc_white_not_hisp', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'annual_payroll', 'fem_num_firms', 'min_num_firms']

test_x = df_2012_test_x2[lst]
full_x = full_test_x2[lst]

print('2016 GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('2016 GRID Search on 2016')
grid_accuracy = evaluate(grid_search, df_2016_test_x, df_2016_test_y)
print('2016 GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_x, full_test_y2)

2016 GRID Search on 2012
mse: 0.04056500505019621
rmse: 0.2014075595656633
mae: 0.16001554872743012
r2: 0.5197801909379347
2016 GRID Search on 2016
mse: 0.0242746630149718
rmse: 0.15580328306865615
mae: 0.11757981080931426
r2: 0.7356740568926636
2016 GRID Search on Full
mse: 0.032419834032584
rmse: 0.18005508610584706
mae: 0.1387976797683722
r2: 0.6434586109921978


In [50]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 1100}

In [51]:
list(zip(lst,list(grid_search.best_estimator_.feature_importances_)))

[('population', 0.04705417481004139),
 ('perc_white', 0.14568738761803365),
 ('perc_black', 0.07277579013663953),
 ('perc_asian', 0.05310104402803184),
 ('perc_white_not_hisp', 0.17229529804812774),
 ('monthly_costs_m', 0.0877446414914472),
 ('monthly_costs_nm', 0.07542278055786204),
 ('perc_bc', 0.0951565223068992),
 ('num_establishments', 0.04972928974990093),
 ('annual_payroll', 0.06072646891692575),
 ('fem_num_firms', 0.06524563602287416),
 ('min_num_firms', 0.07506096631321672)]

In [53]:
# Fit the grid search to the data
grid_search.fit(full_train_x, full_train_y)
grid_search.best_params_

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 35.8min
[Parallel(n_jobs=-1)]: Done 243 out of 243 | elapsed: 56.2min finished
C:\Users\sajor\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 900}

In [54]:
lst = ['population', 'perc_white', 'perc_black', 'perc_white_not_hisp', 'median_oo_value', 'monthly_costs_m', 'monthly_costs_nm', 'perc_bc', 'num_establishments', 'annual_payroll', 'min_num_firms']

test_x = df_2012_test_x2[lst]
test2_x = df_2016_test_x2[lst]

print('FULL GRID Search on 2012')
grid_accuracy = evaluate(grid_search, test_x, df_2012_test_y2)
print('FULL GRID Search on 2016')
grid_accuracy = evaluate(grid_search, test2_x, df_2016_test_y2)
print('FULL GRID Search on Full')
grid_accuracy = evaluate(grid_search, full_test_x, full_test_y)

FULL GRID Search on 2012
mse: 0.031919790509355596
rmse: 0.17866110519459907
mae: 0.13741149221996057
r2: 0.6204370748488228
FULL GRID Search on 2016
mse: 0.023150139867768274
rmse: 0.15215170018034066
mae: 0.11559100224600931
r2: 0.7478065538378071
FULL GRID Search on Full
mse: 0.027534965188561935
rmse: 0.1659366300385841
mae: 0.12650124723298495
r2: 0.6965088840913378


In [55]:
grid_search.best_params_

{'bootstrap': False,
 'max_depth': 100,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 3,
 'n_estimators': 900}

In [56]:
list(zip(lst,list(grid_search.best_estimator_.feature_importances_)))

[('population', 0.07506219888359449),
 ('perc_white', 0.15339720069221613),
 ('perc_black', 0.07028499615253161),
 ('perc_white_not_hisp', 0.1716897021258679),
 ('median_oo_value', 0.06202126553218663),
 ('monthly_costs_m', 0.07258555593753088),
 ('monthly_costs_nm', 0.09739282411537033),
 ('perc_bc', 0.06345892937658154),
 ('num_establishments', 0.07015691427433549),
 ('annual_payroll', 0.08924956676965709),
 ('min_num_firms', 0.07470084614012744)]